![Clarify Logo](https://global-uploads.webflow.com/5e81e464dad44d3a9a32d1f4/5ed10fc3f1ff8467f4466786_logo.svg)
<img src="https://uploads-ssl.webflow.com/5f031b98adc00651e28ef04b/6058a5f7b4c86c42885a2c2c_orchest-logo-no-padding.svg" alt="Orchest Logo" width="200"/>

**Welcome the tutorial about creating and deployment data pipelines combining Clarify and Orchest!**

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/orchest/orchestration.jpg" alt="clarify orchest" width="400">

In this tutorial we combine the power of [Clarify](https://www.clarify.io/) for data exploration, visualization and collaboration across teams with [Orchest](https://www.orchest.io) for data pipelines development and deployment.

# Prerequisites 
This tutorial builds upon the [basic tutorial on using Python with Clarify](https://colab.research.google.com/github/searis/data-science-tutorials/blob/main/tutorials/Introduction.ipynb) and the [forecasting](https://colab.research.google.com/github/searis/data-science-tutorials/blob/main/tutorials/Forecasting.ipynb) tutorial. So we recommend you give a read on those before moving on this tutorial.

## What you need

1. A Clarify account (with admin rights)
2. A working Integration with Signal(s): `clarify-credentials.json` uploaded to the environment running the files
3. A working setup of Orchest:
    - You can create free account and instance on [Orchest Cloud](https://cloud.orchest.io/) and be ready to follow the next steps of the tutorial
    - Alternatively, you can install Orchest for free in your own machine (either locally or in your preferred cloud service)
    - For more details about installation and various possible ways to setup Orchest check the [website](https://www.orchest.io/) and [docs](https://docs.orchest.io/en/latest/)

## What we will do
1. [Initial setup and definitions](#init)
     - [Quickstart](#quickstart)
2. [Pipelines in Orchest](#pipelines)
3. [Read, write to Clarify](#read_write)
4. [Forecast step](#forecast)
5. [Configuring recurring tasks](#cron)
6. [Visualizing the result in Clarify](#visualize)

--- 
Other resources:
* [Clarify API reference](https://docs.clarify.io/reference/http)
* [SDK documentation](https://searis.github.io/pyclarify/)
* [Intro to Python Notebooks](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html#notebook-user-interface)
* [Orchest documentation](https://docs.orchest.io/en/stable/)
* [Merlion - time-series forecast and anomaly detection library](https://opensource.salesforce.com/Merlion/v1.0.1/tutorials.html)

<a name="init"></a>
# Initial setup and definitions

[Clarify](https://www.clarify.io) is a tool for easy data sharing, exploration and collaboration. The [Clarify API](https://docs.clarify.io/docs) simplifies the task of sending timeseries data and metadata to Clarify, allowing for easy integration of multiple sources of data and visualization that can shared on a team, as well as discussion threads, calculated items, thresholds, among many other features. [PyClarify](https://clarify.github.io/pyclarify/user/whatispyclarify.html) is the Python package that facilitates the interaction with the Clarify API, allowing you to read, create and update data and metadata on Clarify. 

[Orchest](https://www.orchest.io) is tool for building data pipeline in a easy way. It consists in a web interface that can accessed either via local installation or via the [Orchest Cloud](https://cloud.orchest.io/). It comes together with a Python interface, that simplifies that tasks of sharing data in a pipeline. Orchest allows you to: define a runtime environment with the necessary packages, visually construct pipelines, write code using JupyterLab (or the native file editor, VSCode or other editor of choice), run subsets of the pipeline, parametrize the pipeline (with pipeline variables and environment variables), run pipelines with a cron-job schedule, among other features. More details about this can be found in the [documentation](https://docs.orchest.io/en/latest/).

Combining the strengths of Orchest and Clarify and their Python interfaces, allows you to easily create data pipelines, that can be readily made available for your whole team. As well as developing data science workflows, that can be scheduled and updated and the results made easily visible for your whole organization. More details about the basic setup of Clarify and PyClarify can be found in the [basic tutorial on using Python with Clarify](https://colab.research.google.com/github/searis/data-science-tutorials/blob/main/tutorials/Introduction.ipynb).

The first step on this setup is then to create an [free account](https://www.clarify.io/signup) on Clarify, and follow the step on the section "Get credentials from Clarify" in the [basic tutorial on using Python with Clarify](https://colab.research.google.com/github/searis/data-science-tutorials/blob/main/tutorials/Introduction.ipynb). The next step is setup a running environment of Orchest. This can be accomplished either by following the [installation](https://docs.orchest.io/en/latest/getting_started/installation.html) steps or setting up an [Orchest Cloud](https://cloud.orchest.io/) account and instance. The remaining part of this tutorial will assume that you have both accounts setup with the correct `clarify-credential.json` file available in the running environment.

<a name="quickstart"></a>
## Quickstart


In order for a quickstart with Clarify and Orchest, we have prepared a GitHub repository that can be readily imported into Orchest. This repository includes a template of a pipeline with steps for reading, forecasting and writing back into Clarify. It also includes the setup for the basic build being used for the project, including all the necessary python packages. We will use this template project as example throughout this tutorial. To use the template you need to:
1. Start your Orchest instance and click in "Projects". 
2. Select the option "Import project"
3. In the field of Git repository, just copy and paste [`https://github.com/clarify/data-science-tutorials-orchest`](https://github.com/clarify/data-science-tutorials-orchest) and in the project name, choose a name following the naming restrictions.
4. If successfuly imported, a pipeline named "Read, Forecast and Write to Clarify" should appear listed in Pipelines. Click on this pipeline and wait for the build to be completed (it can take a couple of minutes, since it is installing multiple Python packages).
5. In order to have access to Clarify API, copy or upload your `clarify-credential.json` to the folder `src`, which is a subfolder of the root folder of the new projected created based on the template provided on GitHub. One way to perform this operation is by going on the `File manager` option in the side bar of Orchest and navigating the folder project.

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/orchest/successfully_imported.png" alt="Import pipeline"  />

The first time that you open the pipeline Orchest will build the environment by installing the different packages necessary (according the the configuration in the `setup.sh` file, that can be also edited). This can take a couple of minutes, but must be done only once. After building the environment you can change the parameters of the steps and test running the the pipeline by selecting the steps and clicking on run selection. Sometimes when running the selection two times in a row you can get memory errors, in this case go to "Settings > Restart memory-server".

You can also schedule a job for running the pipeline multiple times. Click on the side panel and select `Jobs`, then click on the button `Create Job`. You can create a job that run only once, scheduled job at a given time, or a cron job that will repeat according the pre-scheduled configuration. There are preset options for every minute, hourly, daily, weekly and monthly, as well as custom schedule using a [cron expression](https://crontab.guru/). Once this is done you will have a running pipeline and job configuration, and if everything is right, new data being written to Clarify according to the job schedule. You can then on Clarify configure to publish, create timelines and share the data with other members of your organization. More details about this can be found in the [basic tutorial on using Python with Clarify](https://colab.research.google.com/github/searis/data-science-tutorials/blob/main/tutorials/Introduction.ipynb).

<a name="pipelines"></a>
# Pipelines in Orchest

The fundamental element for building and deployment data workflows in Orchest is the [pipeline](https://docs.orchest.io/en/latest/fundamentals/pipelines.html#id1).

<img src="https://docs.orchest.io/en/latest/_images/final-pipeline.png" alt="Orchest pipeline"  />

A pipeline consist of a directed graph of steps, with each step representing a piece of code (for example, one step can be Python script, or a Jupyter notebook). The pipeline can be constructed visually, by connecting nodes to each other, and the input and output of each step can be saved in variables using the [Orchest SDK](https://docs.orchest.io/en/latest/fundamentals/sdk/index.html) (available for Python and R, although this tutorial will focus only on the Python interface).

The main way to pass input/output data between steps is to use the `orchest.get_inputs` and `orchest.output` methods. With `orchest.get_inputs` we obtain a dictionary with keys corresponding to variable names that was passed to the current step by the incoming connections of the step. With `orchest.output(data, name)` you can write a named object, that can be read by the next steps outbound from the current step.

A pipeline is part of a project, that can be composed of multiple ones. 

For an overview of other important concepts of Orchest, check the [overview](https://docs.orchest.io/en/latest/index.html#orchest) and for more detail about passing data between steps in a pipeline check the "Data passing" section on the [quickstart](https://docs.orchest.io/en/latest/fundamentals/data_passing.html).

<a name="read_write"></a>
# Read and write to Clarify 

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/orchest/example_pipeline.png" alt="Tutorial pipeline"  />


The [PyClarify](https://clarify.github.io/pyclarify/user/whatispyclarify.html) package allows us to communicate with the Clarify API. In order to simplify the task of building pipelines and also as an example, we have created the nodes available in `src/` and added them in the example pipeline in the tutorial code. This example pipeline is obtained after succesfully importing the example project following the steps on [Quickstart](#quickstart).

The first step of the pipeline is "Read item configuration" (that uses the code in `src/node_config_read_forecast`), which setups variables using step parameters (can be edited by clicking on the node and editing the values below `Parameters`). The values configured in the step parameters are passed forward to the next nodes that read data from Clarify according to those values. We have the following parameters by default and they should be edited according to the specific item that you have access to and the configuration of the forecast desired. 
```
"parameters": {
                "future": 6,
                "item_id": "c5rtq4jsbu8cohpq1k70",
                "lag_days": 2,
                "name": "item_data_1",
                "time_split": 3
             }
```

The variable `future` defines the number of points to be predicted into the future by the forecast node. The value should be zero or a positive integer bigger than zero. The variable `item_id` defines the item id of interest, it can be obtained by accessing the item viewer or the admin panel on Clarify. The variable `lag_days` is a positive integer greater than zero, and it defines how many days to look into the past to retrieve data (when calling the API we will try to retrieve data from the number of days defined in `lag_days` until now). Notice that the Clarify API has a limit on the time interval of data that we can retrieve with one call to the API (it is possible to extend that limit by making multiple calls to the API). The variable `time_split` defines how many points to use in the training / testing time split of the data, and it needs to be also a positive number. When training the forecast model we a small portion of the most recent observations of the time series as testing, so that we can validate and visualize the performance of the forecast. The variable `name` is simply used to name the item we are retrieving the data.

The node named "Write data back to Clarify" (that used the code in `src/node_write_items`) receives data from the pipeline and proceeds to use PyClarify to send a request to create a signal (adding metadata to it, in case there it is provided) and write the data in Clarify. This node expect a named variable called `clfy_dict`, which is a dictionary with each key being a `signal_id`, and the values for each key being another dictionary with mandatory fields `name`, `labels`, `times`, `series` and the optional field `kargs` consisting on other metadata fields. The node creates a signal using `signal_id`, `name`, `labels` and `kargs` (if present) to fill the metadata, and `times` and `series` for the data. After calling the Clarify API a response is generated and can be read in the logs.

The pipeline also include an extra node "Read items" that simply uses the information from "Read item configuration" to retrieve data from the API, but does not do any further processing of that data. This node can be used for passing the data for further processing, or it can be customized by editing the file `src/node_read_items.py`.

<a name="forecast"></a>
# Forecast step

The "Forecast with lag and future" step uses the code in `src/node_read_item_forecast.py` and receives the configurations from the previous step "Read item configuration" discussed in the previous section. The step consist in code that reads data from Clarify, prepare the data and uses that data to forecast future values of the time series, generating a prediction, as well as upper and lower bound for the uncertainty associated with the prediction. The prediction, upper and lower bound of the uncertainty of the prediction are then aggregated in a dictionary and past to the next steps using the named variable `clfy_dict`, following the convention of the node "Write data back to Clarify". The generated signals will be named as `"Forecast {signal_name}"` for the main prediction, with the addition of `" upper bound"` and `" lower bound"` suffix for the upper and lower bound of the uncertainty estimates.  In general the following metadata will be included:

```    
    {
            "name" : name,
            "labels" : {"source":["Orchest pipelines"], "original_id":[original_id]},
            "times" : times,
            "series" : values,
            "kargs" : {"sourceType" : "prediction",
                        "data-source": ["Orchest"],
                        "description" : f"Forecast for {name}"
                      }
    }
```

The following diagram represent the main parameters and how they are mapped into segments of the timeseries. The forecasting algorithm process the timeseries retrieving data from `lag_days` until the now, then split a part of the retrieved data for training and testing using the `time_split` parameters, and after training on past data, predicts the future values generating predictions with the `future` parameter controlling how far into to future to predict.

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/orchest/forecast_lag_future.png" alt="Forecast lag and future"  />

The forecast is created using the [`merlion`](https://github.com/salesforce/Merlion), which is a library that encapsulates multiple algorithms and methods for timeseries data analysis. In this case the Prophet algorithm is used with the standard configuration. More levels of customization can be achieved by editing the file `src/node_read_item_forecast.py`, for example testing different forecasting methods. The implementation follows closely the [forecasting](https://colab.research.google.com/github/searis/data-science-tutorials/blob/main/tutorials/Forecasting.ipynb) tutorial, which can be consulted for more details as well.

<a name="cron"></a>
# Configuring recurring tasks



Once we have a pipeline that is running without errors, we can configure jobs in order to run the pipeline automatically following a certain schedule. This feature can be accessed by clicking on the left side panel, and selecting `Jobs`. The setup of a job includes options about scheduling, parameters, environment variables and you can access the different runs of the jobs and the associated logs. 

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/orchest/jobs.png" alt="Jobs"  />

In order to continously run the pipeline, leading to automatically processing the data and adding new predictions to Clarify, we must select the option `Cron job`. A cron job have a predetermined schedule for re-runs, and in the UI interface you can options to choose from every minute, hourly, daily, weekly or monthly. Alternatively you can write your custom definition of the time interval to re-run the job using a `cron expression`. It is beyond the scope of this tutorial to teach cron expressions, nevertheless there are some online services like [crontab.guru](https://crontab.guru/) that can be helpful when creating a custom expression. More information about jobs on Orchest can be found in the [official documentation about jobs](https://docs.orchest.io/en/stable/fundamentals/jobs.html).

<a name="visualize"></a>
# Visualizing the result in Clarify

Finally with our pipeline running and schedule to re-run on a given time period, we can visualize the data on Clarify, communicate with other team member about it and have it updated automatically with new results from the pipeline runs. The combination of Orchest with Clarify creates a powerful toolset for data analysis, visualization, as well as a simple way to deploy models that will be continously generating new results.

The signals generated by running the example pipeline can be found in the admin panel, under the same integration from which we obtained the `clarify-credentials.json` file. The name of the signal will have the prefix "Forecast " followed by the name of the signal and the suffices " upper bound" and " lower bound" in case of the uncertainty estimates. You can choose to publish the signals, visualize them individually, create custom timelines with them and share with other team members. In case we put all the visualization of the original timeseries as well the forecast results, we can create a timeline that could look like this example below.

<img src="https://raw.githubusercontent.com/clarify/data-science-tutorials/main/media/forecast/clarify_forecast_comment.png" alt="Visualization of forecast in the timeline in Clarify" style="width: 70%;"  />

More details about publishing signals, visualizations and sharing of timelines can be found in the [basic tutorial on using Python with Clarify](https://colab.research.google.com/github/searis/data-science-tutorials/blob/main/tutorials/Introduction.ipynb) and the [forecasting](https://colab.research.google.com/github/searis/data-science-tutorials/blob/main/tutorials/Forecasting.ipynb) tutorial.